## import previous file

In [124]:
import pandas as pd
from pathlib import Path

p = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv3_99_23.parquet")
df_my_cov_aligned_short = pd.read_parquet(p)  # uses pyarrow/fastparquet if available
print(df_my_cov_aligned_short.shape)
df_my_cov_aligned_short.head()


(128809, 66)


,SEQN,SDDSRVYR,sdmvpsu,sdmvstra,RIDAGEYR,SEX,RACE,re2,re,household_size,...,SNAP_indiv_only,SNAP_indiv_plus_singleton,bmi,HOQ065_structural_missing,household_size_structural_missing,chol_rx_structural_missing,RIAGENDR,drinking,alcg2,perE_alco
0,1,1.00,1,5,2.00,F,4.00,4.00,Other Hispanic,3,...,0,0,NaN,False,False,False,2.00,NaN,<NA>,0.00
1,2,1.00,3,1,77.00,M,3.00,3.00,Mexican American,1,...,<NA>,<NA>,NaN,False,False,False,1.00,0.07,2,0.00
2,3,1.00,2,7,10.00,F,3.00,3.00,Mexican American,4,...,<NA>,<NA>,NaN,False,False,False,2.00,NaN,<NA>,0.00
3,4,1.00,1,2,1.00,M,4.00,4.00,Other Hispanic,7,...,<NA>,<NA>,NaN,False,False,False,1.00,NaN,<NA>,0.00
4,5,1.00,2,8,49.00,M,3.00,3.00,Mexican American,3,...,<NA>,<NA>,NaN,False,False,False,1.00,1.71,2,9.10


## Check met_hr

In [125]:
df_my_cov_aligned_short.columns

Index(['SEQN', 'SDDSRVYR', 'sdmvpsu', 'sdmvstra', 'RIDAGEYR', 'SEX', 'RACE',
       're2', 're', 'household_size', 'DMDHHSIZ', 'EDU', 'pir', 'SMK_AVG',
       'SMK', 'met_hr', 'SMK_STATUS', 'CIGS_PER_DAY', 'PACK_YEARS',
       'FORMER_SMOKER', 'METSCORE', 'LTPA', 'bmi_cat', 'BMI_CLAS', 'DIABE',
       'HYPERTEN', 'chol_rx', 'CVD', 'cancer', 'probable_depression',
       'ahei_total', 'unemployment2', 'ins', 'HOQ065', 'marriage', 'SNAP',
       'FS', 'WTINT2YR', 'WTMEC2YR', 'WTSAF2YR', 'WTINT4YR', 'WTMEC4YR',
       'WTPH2YR', 'WTINTPRP', 'WTMECPRP', 'WTSAFPRP', 'wt_int', 'wt_mec',
       'wt_fasting', 'wt_phlebotomy', 'marriage_prev', 'marriage_label',
       'marriage3', 'SNAP_src', 'SNAP_bin', 'SNAP_src_rank', 'SNAP_indiv_only',
       'SNAP_indiv_plus_singleton', 'bmi', 'HOQ065_structural_missing',
       'household_size_structural_missing', 'chol_rx_structural_missing',
       'RIAGENDR', 'drinking', 'alcg2', 'perE_alco'],
      dtype='object')

In [126]:
df_my_cov_aligned_short[["chol_rx", "met_hr", "METSCORE", "LTPA"]]

,chol_rx,met_hr,METSCORE,LTPA
0,0,NaN,NaN,NaN
1,0,60.00,60.00,0.00
2,0,NaN,NaN,NaN
3,0,NaN,NaN,NaN
4,0,1920.00,1920.00,41.07
...,...,...,...,...
128804,<NA>,NaN,NaN,NaN
128805,1,180.00,180.00,3.00
128806,0,180.00,180.00,3.00
128807,0,840.00,840.00,14.00


In [127]:
df = df_my_cov_aligned_short

# structural missing where module absent / not merged
df["chol_rx_structural_missing"] = False
df.loc[df["SDDSRVYR"].isin([12.0, 66.0]) & df["chol_rx"].isna(), "chol_rx_structural_missing"] = True


In [128]:
import numpy as np

df = df_my_cov_aligned_short

# Hard cap to remove extreme glitches (tune as needed)
# Example: cap at 100,000 (≈ 32 hours/week all year if in minutes) — adjust to your definition
cap = 100_000
df["met_hr_clean"] = df["met_hr"].where((df["met_hr"] >= 0) & (df["met_hr"] <= cap))

# Optional: winsorize top 0.5% within each cycle to reduce tail influence
def winsorize_by_cycle(s, upper=0.995):
    q = s.quantile(upper)
    return s.clip(upper=q)

df["met_hr_win"] = (
    df.groupby("SDDSRVYR", group_keys=False)["met_hr_clean"]
      .apply(lambda s: winsorize_by_cycle(s, 0.995))
)


In [129]:
# chol_rx % by category (keeps NAs visible)
chol_rx_dist = (
    df.groupby("SDDSRVYR")["chol_rx"]
      .value_counts(normalize=True, dropna=False)
      .rename("pct")
      .mul(100)
      .round(2)
      .unstack(fill_value=0)
)

# met_hr (clean/winsorized)
met_stats = (
    df.groupby("SDDSRVYR")["met_hr_win"]
      .agg(count="count", mean="mean", std="std", min="min", max="max")
      .round(2)
)

print("chol_rx distribution by cycle (%):")
print(chol_rx_dist)
print("\nmet_hr (winsorized) summary by cycle:")
print(met_stats)


chol_rx distribution by cycle (%):
chol_rx   0     1      <NA>
SDDSRVYR                   
1.00     95.77  4.23   0.00
2.00     94.45  5.55   0.00
3.00     92.74  7.26   0.00
4.00     92.53  7.47   0.00
5.00     88.24 11.76   0.00
6.00     88.31 11.69   0.00
7.00     88.72 11.28   0.00
8.00     87.95 12.05   0.00
9.00     88.27 11.73   0.00
10.00    85.76 14.24   0.00
12.00    53.12 17.82  29.06
66.00     0.00  0.00 100.00

met_hr (winsorized) summary by cycle:
          count    mean     std  min      max
SDDSRVYR                                     
1.00       4880 3376.14 6098.91 0.00 36675.60
2.00       5411 3497.97 6073.79 0.00 35514.00
3.00       5041 3295.93 5903.89 0.00 35904.00
4.00       4979 3556.04 6093.40 0.00 34680.00
5.00       5935 3727.17 6454.69 0.00 36960.00
6.00       6218 3141.25 5311.27 0.00 30692.80
7.00       5560 3030.38 5068.51 0.00 30480.00
8.00       5769 3112.37 5325.17 0.00 32097.60
9.00       5719 3635.10 6044.94 0.00 34560.00
10.00      5569 4156.74 6946

In [130]:
# Ensure SEQN is numeric Int64 (often fixes "not found" issues)
if df_my_cov_aligned_short['SEQN'].dtype != 'Int64':
    df_my_cov_aligned_short['SEQN'] = (
        pd.to_numeric(df_my_cov_aligned_short['SEQN'], errors='coerce').astype('Int64')
    )

# Pick a small set of useful columns; add/remove as you like
cols = [c for c in [
    'SEQN','SDDSRVYR','met_hr','SMK','ALCG2','WTINT2YR','WTMEC2YR','BMI','bmic'
] if c in df_my_cov_aligned_short.columns]

# Pull and sort
check = (df_my_cov_aligned_short
         .loc[df_my_cov_aligned_short['SEQN'].isin([31131, 31132]), cols]
         .sort_values(['SEQN']))

# If multiple rows per SEQN, show them all
display(check)

# Helpful sanity prints
print("Rows found:", len(check))
print("Any met_hr missing?:", check['met_hr'].isna().any() if 'met_hr' in check else "met_hr column not in data")


,SEQN,SDDSRVYR,met_hr,SMK,WTINT2YR,WTMEC2YR
31130,31131,4.00,720.00,NEVER,26457.71,26770.58
31131,31132,4.00,5540.00,NEVER,32961.51,35315.54


Rows found: 2
Any met_hr missing?: False


## Correct met_hr

In [131]:
import pandas as pd
import numpy as np
from pathlib import Path

# =========================================================
# Settings
# =========================================================
# Paths (edit if your PAQ_L filename differs)
P2017_2020_PATH = Path("/Users/dengshuyue/Desktop/SDOH/analysis/data/cov/pa/P_PAQ.xpt")
PAQ_L_PATH      = Path("/Users/dengshuyue/Desktop/SDOH/analysis/data/cov/pa/PAQ_L.xpt")

# Choose whether to include work & transport domains for P_PAQ (Appendix 1)
# False/False = LTPA-only (matches your prior convention)
INCLUDE_WORK_IN_P_PAQ      = False
INCLUDE_TRANSPORT_IN_P_PAQ = False

# Hard ceilings to kill biologically impossible reports
MAX_MET_HR_PER_WEEK = 100.0                    # final result cap
MAX_MET_MIN_PER_WEEK = MAX_MET_HR_PER_WEEK*60  # 6000
MAX_MINUTES_PER_OCCASION = 1440                # < 24h
MAX_DAYS_PER_WEEK = 7

# =========================================================
# Load helpers
# =========================================================
def read_xpt(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Missing: {path}")
    df = pd.read_sas(path, format="xport", encoding="latin1")
    df.columns = [c.upper() for c in df.columns]
    if "SEQN" not in df.columns:
        raise KeyError(f"{path.name} lacks SEQN")
    df["SEQN"] = pd.to_numeric(df["SEQN"], errors="coerce").astype("Int64")
    return df

def _num(s):
    return pd.to_numeric(s, errors="coerce")

def _clean_days(s):
    s = _num(s)
    return s.mask(s.isin([77, 99]) | (s < 0) | (s > MAX_DAYS_PER_WEEK))

def _clean_min(s):
    s = _num(s)
    return s.mask(s.isin([7777, 9999]) | (s <= 0) | (s > MAX_MINUTES_PER_OCCASION))

def _map_unit(u):
    # robust parser for PAQ_L units: Day/Week/Month/Year (D/W/M/Y)
    u = pd.Series(u).astype(str).str.strip().str.upper().str.replace(r'[^A-Z]', '', regex=True)
    lead = u.str.slice(0, 1)
    return lead.where(lead.isin(list("DWMY")), np.nan)

def _times_per_week(q, u):
    q = _num(q).mask(_num(q).isin([7777, 9999]))
    unit = _map_unit(u)
    mult = np.where(unit == "D", 7.0,
           np.where(unit == "W", 1.0,
           np.where(unit == "M", 52.0/12.0,
           np.where(unit == "Y", 52.0, np.nan))))
    wk = pd.to_numeric(q, errors="coerce") * pd.to_numeric(mult, errors="coerce")
    # leisure-time frequency guard (soft edits mention >28/wk as extreme)
    return wk.mask((wk < 0) | (wk > 28))

# =========================================================
# Derivations
# =========================================================
def derive_from_P_PAQ(paq: pd.DataFrame,
                      include_work: bool = False,
                      include_transport: bool = False) -> pd.DataFrame:
    """
    NHANES P_PAQ (2017–Mar 2020). Uses Appendix 1 MET weights:
      Vigorous work: PAD615 * 8.0
      Moderate work: PAD630 * 4.0
      Transport walk/bike: PAD645 * 4.0
      Vigorous leisure: PAD660 * 8.0
      Moderate leisure: PAD675 * 4.0
    Minutes are per *typical day*; multiply by *days/week*.
    """
    df = paq.copy()
    df.columns = [c.upper() for c in df.columns]

    # Any-activity flags (may be used to set explicit zeros)
    any_vw = _num(df.get("PAQ605"))
    any_mw = _num(df.get("PAQ620"))
    any_tr = _num(df.get("PAQ635"))
    any_vr = _num(df.get("PAQ650"))
    any_mr = _num(df.get("PAQ665"))

    # Days (1–7) & minutes/occasion (<=1440)
    vw_days = _clean_days(df.get("PAQ610")); vw_min = _clean_min(df.get("PAD615"))
    mw_days = _clean_days(df.get("PAQ625")); mw_min = _clean_min(df.get("PAD630"))
    tr_days = _clean_days(df.get("PAQ640")); tr_min = _clean_min(df.get("PAD645"))
    vr_days = _clean_days(df.get("PAQ655")); vr_min = _clean_min(df.get("PAD660"))
    mr_days = _clean_days(df.get("PAQ670")); mr_min = _clean_min(df.get("PAD675"))

    # Weekly minutes per domain
    vw = (vw_days.fillna(0) * vw_min.fillna(0)) if include_work else 0
    mw = (mw_days.fillna(0) * mw_min.fillna(0)) if include_work else 0
    tr = (tr_days.fillna(0) * tr_min.fillna(0)) if include_transport else 0
    vr =  vr_days.fillna(0) *  vr_min.fillna(0)  # leisure always included
    mr =  mr_days.fillna(0) *  mr_min.fillna(0)

    # Apply MET weights
    met_min = (8.0*vw + 4.0*mw + 4.0*tr + 8.0*vr + 4.0*mr).astype("float64")

    # Completion logic: require days & minutes for at least one included domain
    has_vw = include_work and (vw_days.notna() & vw_min.notna())
    has_mw = include_work and (mw_days.notna() & mw_min.notna())
    has_tr = include_transport and (tr_days.notna() & tr_min.notna())
    has_vr = (vr_days.notna() & vr_min.notna())
    has_mr = (mr_days.notna() & mr_min.notna())

    domains_any = has_vr | has_mr | has_vw | has_mw | has_tr

    # Explicit zeros if all included domains answered "No"
    all_no = (
        ((any_vr == 2) & (any_mr == 2)) &
        ((any_vw == 2) | (not include_work)) &
        ((any_mw == 2) | (not include_work)) &
        ((any_tr == 2) | (not include_transport)) #~include_transport 
    )

    met_min = met_min.where(domains_any, np.nan).where(~all_no, 0)
    met_min = met_min.mask(met_min > MAX_MET_MIN_PER_WEEK, np.nan)

    out = df[["SEQN"]].copy()
    out["METSCORE"] = met_min
    out["LTPA"] = met_min / 60.0
    return out

def derive_from_PAQ_L(paq_l: pd.DataFrame) -> pd.DataFrame:
    """
    NHANES PAQ_L (Aug 2021–Aug 2023). Leisure only:
      Moderate: (PAD790Q, PAD790U) freq × PAD800 minutes/occ × 4.0
      Vigorous: (PAD810Q, PAD810U) freq × PAD820 minutes/occ × 8.0
    """
    df = paq_l.copy()
    df.columns = [c.upper() for c in df.columns]

    mod_wk   = _times_per_week(df.get("PAD790Q"), df.get("PAD790U"))
    vig_wk   = _times_per_week(df.get("PAD810Q"), df.get("PAD810U"))
    mod_minO = _clean_min(df.get("PAD800"))
    vig_minO = _clean_min(df.get("PAD820"))

    mod_met_min = 4.0 * mod_wk.fillna(0) * mod_minO.fillna(0)
    vig_met_min = 8.0 * vig_wk.fillna(0) * vig_minO.fillna(0)
    met_min = (mod_met_min + vig_met_min).astype("float64")

    # require both freq & duration for each domain; allow explicit zeros
    has_mod = mod_wk.notna() & mod_minO.notna()
    has_vig = vig_wk.notna() & vig_minO.notna()
    has_any = has_mod | has_vig

    zero_mod = (_num(df.get("PAD790Q")) == 0)
    zero_vig = (_num(df.get("PAD810Q")) == 0)
    true_zero = (zero_mod & zero_vig)

    met_min = met_min.where(has_any, np.nan).where(~true_zero, 0)
    met_min = met_min.mask(met_min > MAX_MET_MIN_PER_WEEK, np.nan)

    out = df[["SEQN"]].copy()
    out["METSCORE"] = met_min
    out["LTPA"] = met_min / 60.0
    return out

# =========================================================
# Load, compute, merge
# =========================================================
paq_p = read_xpt(P2017_2020_PATH)
paq_l = read_xpt(PAQ_L_PATH)

ltpa_p = derive_from_P_PAQ(paq_p,
                           include_work=INCLUDE_WORK_IN_P_PAQ,
                           include_transport=INCLUDE_TRANSPORT_IN_P_PAQ)
ltpa_l = derive_from_PAQ_L(paq_l)

pa_all = (pd.concat([ltpa_p, ltpa_l], ignore_index=True)
            .dropna(subset=["SEQN"])
            .groupby("SEQN", as_index=False)
            .agg({"METSCORE":"max","LTPA":"max"}))

print("PA rows (unique SEQN):", len(pa_all))

# Merge into your working dataframe
df = df_my_cov_aligned_short.copy()
df["SEQN"] = pd.to_numeric(df["SEQN"], errors="coerce").astype("Int64")
merged = df.merge(pa_all, on="SEQN", how="left", suffixes=("", "_fromPAQ"))

# New recomputed met_hr (hours/week), with fallback to old columns if PAQ is missing
met_min = pd.to_numeric(merged["METSCORE"], errors="coerce")
merged["met_hr_recalc"] = (met_min / 60.0)
merged["met_hr_recalc_from"] = np.where(merged["met_hr_recalc"].notna(), "from_new_PAQ", pd.NA)

fallback = merged["met_hr_recalc"].isna()
if fallback.any():
    old_met = pd.to_numeric(merged.get("met_hr"), errors="coerce")
    old_ltpa = pd.to_numeric(merged.get("LTPA"), errors="coerce")
    cand = pd.Series(np.nan, index=merged.index, dtype="float64")
    m = fallback & old_met.notna() & (old_met.between(0, 200));        cand[m] = old_met[m]           # hours
    m = fallback & old_met.notna() & (old_met > 200);                   cand[m] = old_met[m] / 60.0    # minutes
    m = fallback & old_ltpa.notna() & (old_ltpa.between(0, 200));       cand[m] = old_ltpa[m]          # hours
    m = fallback & old_ltpa.notna() & (old_ltpa > 200);                 cand[m] = old_ltpa[m] / 60.0   # minutes
    merged.loc[fallback, "met_hr_recalc"] = cand
    merged.loc[fallback & cand.notna(), "met_hr_recalc_from"] = "fallback_old_cols"

# Final cap (impossible totals → NA)
merged.loc[(merged["met_hr_recalc"] < 0) | (merged["met_hr_recalc"] > MAX_MET_HR_PER_WEEK), "met_hr_recalc"] = np.nan

# Quick sanity summaries
print("Non-missing met_hr_recalc:", (merged["met_hr_recalc"].notna().mean()*100).round(1), "%")
summary = (merged.dropna(subset=["met_hr_recalc"])
                 .groupby("SDDSRVYR")["met_hr_recalc"]
                 .agg(mean="mean", median="median",
                      p90=lambda s: s.quantile(0.9), max="max", count="count")
                 .reset_index()
                 .sort_values("SDDSRVYR"))
pd.set_option("display.float_format", "{:.2f}".format)
display(summary.head(20))

# Hand back to your working variable
df_my_cov_aligned_short = merged
print("✓ met_hr_recalc merged into df_my_cov_aligned_short")

# Optional: if you want to overwrite your working met_hr column now:
# df_my_cov_aligned_short["met_hr_old"] = df.get("met_hr")
# df_my_cov_aligned_short["met_hr"] = df_my_cov_aligned_short["met_hr_recalc"]


PA rows (unique SEQN): 17846
Non-missing met_hr_recalc: 48.3 %


,SDDSRVYR,mean,median,p90,max,count
0,1.00,18.96,8.00,60.00,100.00,4054
1,2.00,19.29,8.00,58.93,100.00,4437
2,3.00,18.29,8.00,56.00,100.00,4179
3,4.00,19.96,9.00,60.00,100.00,4080
4,5.00,19.46,8.00,62.00,100.00,4797
5,6.00,20.21,9.33,60.00,100.00,5223
6,7.00,21.47,12.00,62.00,100.00,4738
7,8.00,20.47,10.00,60.00,100.00,4869
8,9.00,20.67,9.33,62.67,100.00,4637
9,10.00,20.83,10.00,60.67,100.00,4412


✓ met_hr_recalc merged into df_my_cov_aligned_short


In [132]:
# Tag each SEQN by which PA file provided its PAQ values
pa_all_src = (pd.concat([
    ltpa_p.assign(paq_source="P_PAQ (2017–Mar 2020)"),
    ltpa_l.assign(paq_source="PAQ_L (2021–2023)"),
], ignore_index=True)
 .dropna(subset=["SEQN"])
 .groupby("SEQN", as_index=False)
 .agg({"paq_source":"first"}))

# Join to your core frame to see how SDDSRVYR maps to the source
tmp = (df_my_cov_aligned_short[["SEQN","SDDSRVYR"]]
       .merge(pa_all_src, on="SEQN", how="left"))

# Cross-tab: for each SDDSRVYR, how many rows came from each PA file?
print(tmp.groupby(["SDDSRVYR","paq_source"]).size()
        .reset_index(name="n")
        .sort_values(["SDDSRVYR","paq_source"]))


   SDDSRVYR             paq_source     n
0     12.00      PAQ_L (2021–2023)  8153
1     66.00  P_PAQ (2017–Mar 2020)  9693


In [134]:
df_my_cov_aligned_short.shape

(128809, 72)

#### check what is driving low met_hr for 2017-2020

In [135]:
import pandas as pd
import numpy as np

# 1) Reliable MET_HR (hours/week)
d = df_my_cov_aligned_short.copy()

def pick_met_hours(df: pd.DataFrame) -> pd.Series:
    # Prefer recomputed hours if present
    if 'met_hr_recalc' in df.columns:
        x = pd.to_numeric(df['met_hr_recalc'], errors='coerce')
        if x.notna().any():
            return x

    # Fallbacks: interpret >200 as minutes
    met  = pd.to_numeric(df.get('met_hr'), errors='coerce') if 'met_hr' in df.columns else pd.Series(np.nan, index=df.index)
    ltpa = pd.to_numeric(df.get('LTPA'),   errors='coerce') if 'LTPA'   in df.columns else pd.Series(np.nan, index=df.index)

    cand = pd.Series(np.nan, index=df.index, dtype='float64')
    m = met.notna() & met.between(0, 200);            cand[m] = met[m]           # hours
    m = met.notna() & (met > 200);                    cand[m] = met[m] / 60.0    # minutes -> hours
    m = cand.isna() & ltpa.notna() & ltpa.between(0, 200);  cand[m] = ltpa[m]
    m = cand.isna() & ltpa.notna() & (ltpa > 200);         cand[m] = ltpa[m] / 60.0
    return cand

d['MET_HR'] = pick_met_hours(d)
d['SDDSRVYR'] = pd.to_numeric(d['SDDSRVYR'], errors='coerce')

# 2) Keep only cycles 12 and 66
sub = d.loc[d['SDDSRVYR'].isin([12, 66]), ['SDDSRVYR','MET_HR']].dropna()
sub['active'] = sub['MET_HR'] > 0

# 3) Compute stats with a tiny loop (no pivot/melt)
rows = []
for cyc, g in sub.groupby('SDDSRVYR'):
    s = g['MET_HR']
    a = g['active']
    rows.append({
        'SDDSRVYR': float(cyc),
        'n': int(len(s)),
        '% active': a.mean()*100,
        '% zero': (~a).mean()*100,
        'mean': s.mean(),
        'median': s.median(),
        'p90': s.quantile(0.90),
        'mean_if_active': s[a].mean() if a.any() else np.nan,
        'median_if_active': s[a].median() if a.any() else np.nan,
        'max': s.max(),
    })

summary = pd.DataFrame(rows).sort_values('SDDSRVYR')
pd.set_option('display.float_format', '{:.2f}'.format)
print("Cycle 66 vs 12 — % active and distribution (unweighted):")
display(summary)

# 4) % active mini-table + delta
pct_active = summary[['SDDSRVYR','% active']].set_index('SDDSRVYR').round(2)
print("\n% active by cycle (MET-h/week > 0):")
display(pct_active)

idx = pct_active.index.astype(float)
if (idx==12.0).any() and (idx==66.0).any():
    delta = float(pct_active.loc[12.0, '% active'] - pct_active.loc[66.0, '% active'])
    print(f"Δ % active (12 − 66): {delta:.2f} percentage points")


Cycle 66 vs 12 — % active and distribution (unweighted):


,SDDSRVYR,n,% active,% zero,mean,median,p90,mean_if_active,median_if_active,max
0,12.00,7325,78.23,21.77,21.17,14.00,56.00,27.06,20.00,100.00
1,66.00,9469,46.45,53.55,11.73,0.00,40.00,25.25,18.00,100.00



% active by cycle (MET-h/week > 0):


,% active
SDDSRVYR,
12.00,78.23
66.00,46.45


Δ % active (12 − 66): 31.78 percentage points


#### check weighted met_hr results

In [136]:
import numpy as np
import pandas as pd

# ===== 0) Source & MET column (hours/week)
df = df_my_cov_aligned_short.copy()
df['SDDSRVYR'] = pd.to_numeric(df['SDDSRVYR'], errors='coerce')

# Prefer recomputed MET hours; fallback to met_hr/LTPA if needed
def pick_met_hours(df):
    if 'met_hr_recalc' in df.columns:
        x = pd.to_numeric(df['met_hr_recalc'], errors='coerce')
        if x.notna().any():
            return x
    met  = pd.to_numeric(df.get('met_hr'), errors='coerce') if 'met_hr' in df.columns else pd.Series(np.nan, index=df.index)
    ltpa = pd.to_numeric(df.get('LTPA'),   errors='coerce') if 'LTPA'   in df.columns else pd.Series(np.nan, index=df.index)
    cand = pd.Series(np.nan, index=df.index, dtype='float64')
    m = met.notna() & met.between(0, 200);            cand[m] = met[m]           # hours
    m = met.notna() & (met > 200);                    cand[m] = met[m] / 60.0    # minutes -> hours
    m = cand.isna() & ltpa.notna() & ltpa.between(0, 200);  cand[m] = ltpa[m]
    m = cand.isna() & ltpa.notna() & (ltpa > 200);         cand[m] = ltpa[m] / 60.0
    return cand

df['MET_HR'] = pick_met_hours(df)

# ===== 1) Build weight per row: WTINT4YR for 66, else WTINT2YR
w4 = pd.to_numeric(df.get('WTINT4YR'), errors='coerce')  # combined weight for 2017–Mar 2020
w2 = pd.to_numeric(df.get('WTINT2YR'), errors='coerce')  # standard interview weight
df['W'] = np.where(df['SDDSRVYR'].astype(float).eq(66.0), w4, w2)

# ===== 2) Weighted helpers
def wavg(x, w):
    m = x.notna() & w.notna() & (w > 0)
    return np.average(x[m], weights=w[m]) if m.any() else np.nan

# ===== 3) Compute stats for all cycles 1–12 and 66
include_cycles = set(range(1,13)) | {66}
rows = []
for cyc, g in df.loc[df['SDDSRVYR'].isin(include_cycles), ['SDDSRVYR','MET_HR','W']].groupby('SDDSRVYR'):
    g = g.dropna(subset=['W'])  # must have the correct weight for that row
    met = pd.to_numeric(g['MET_HR'], errors='coerce')
    w   = pd.to_numeric(g['W'], errors='coerce')
    active = met > 0
    rows.append({
        'SDDSRVYR': int(cyc),
        'n (with weight)': int((met.notna() & w.notna() & (w>0)).sum()),
        '% active (weighted)': wavg(active.astype(float), w) * 100,
        'mean MET-h/wk (weighted)': wavg(met, w),
        'mean_if_active (weighted)': wavg(met.where(active), w),
    })

weighted_by_cycle = (pd.DataFrame(rows)
                       .sort_values('SDDSRVYR')
                       .set_index('SDDSRVYR')
                       .round(2))

print("Weighted leisure-only MET-h/week by cycle (WTINT4YR for cycle 66; WTINT2YR otherwise):")
display(weighted_by_cycle)

# Optional: quick diff 12 vs 66
if 12 in weighted_by_cycle.index and 66 in weighted_by_cycle.index:
    d_pct = float(weighted_by_cycle.loc[12, '% active (weighted)'] - weighted_by_cycle.loc[66, '% active (weighted)'])
    d_mean = float(weighted_by_cycle.loc[12, 'mean MET-h/wk (weighted)'] - weighted_by_cycle.loc[66, 'mean MET-h/wk (weighted)'])
    print(f"\nΔ % active (12 − 66), weighted: {d_pct:.2f} pp")
    print(f"Δ mean MET-h/wk (12 − 66), weighted: {d_mean:.2f}")


Weighted leisure-only MET-h/week by cycle (WTINT4YR for cycle 66; WTINT2YR otherwise):


,n (with weight),% active (weighted),mean MET-h/wk (weighted),mean_if_active (weighted)
SDDSRVYR,,,,
1,4054,38.27,21.25,31.56
2,4437,37.83,21.11,31.50
3,4179,37.53,19.49,30.26
4,4080,38.46,20.85,31.47
5,4797,39.52,22.49,32.39
6,5223,43.44,23.07,32.31
7,4738,44.85,23.47,32.11
8,4869,43.78,21.87,30.92
9,4637,43.17,23.68,32.49



Δ % active (12 − 66), weighted: 15.50 pp
Δ mean MET-h/wk (12 − 66), weighted: 8.62


In [139]:
# only remove PA intermediates if they exist; leave all other columns untouched
to_drop = [c for c in ["met_hr_clean","met_hr_win","met_hr_recalc"]
           if c in df_my_cov_aligned_short.columns]

df_my_cov_aligned_short = df_my_cov_aligned_short.drop(columns=to_drop)


In [140]:
df_my_cov_aligned_short.columns

Index(['SEQN', 'SDDSRVYR', 'sdmvpsu', 'sdmvstra', 'RIDAGEYR', 'SEX', 'RACE',
       're2', 're', 'household_size', 'DMDHHSIZ', 'EDU', 'pir', 'SMK_AVG',
       'SMK', 'met_hr', 'SMK_STATUS', 'CIGS_PER_DAY', 'PACK_YEARS',
       'FORMER_SMOKER', 'METSCORE', 'LTPA', 'bmi_cat', 'BMI_CLAS', 'DIABE',
       'HYPERTEN', 'chol_rx', 'CVD', 'cancer', 'probable_depression',
       'ahei_total', 'unemployment2', 'ins', 'HOQ065', 'marriage', 'SNAP',
       'FS', 'WTINT2YR', 'WTMEC2YR', 'WTSAF2YR', 'WTINT4YR', 'WTMEC4YR',
       'WTPH2YR', 'WTINTPRP', 'WTMECPRP', 'WTSAFPRP', 'wt_int', 'wt_mec',
       'wt_fasting', 'wt_phlebotomy', 'marriage_prev', 'marriage_label',
       'marriage3', 'SNAP_src', 'SNAP_bin', 'SNAP_src_rank', 'SNAP_indiv_only',
       'SNAP_indiv_plus_singleton', 'bmi', 'HOQ065_structural_missing',
       'household_size_structural_missing', 'chol_rx_structural_missing',
       'RIAGENDR', 'drinking', 'alcg2', 'perE_alco', 'METSCORE_fromPAQ',
       'LTPA_fromPAQ', 'met_hr_recalc_fr

In [141]:
df_my_cov_aligned_short.shape

(128809, 69)

## check missingness again 

In [192]:
df = df_my_cov_aligned_short

# use every column except the grouper
# ignore PACK_YEARS, CIGS_PER_DAY, probable_depression and etc as missing naturally 
# ignore dulicate safe saved old column name

exclude = {"SDDSRVYR","CIGS_PER_DAY","PACK_YEARS","probable_depression","wt_phlebotomy", "WTSAFPRP",
           "WTINT2YR", "WTMEC2YR", "WTPH2YR", "WTSAF2YR", "WTMEC4YR", "WTINTPRP", "WTMECPRP", "WTINT4YR",
           "SNAP", "SNAP_src", "SNAP_bin", "SNAP_src_rank", "bmi", "RIAGENDR",
           "SNAP_indiv_only","FS", "ahei_total", "HOQ065", "marriage_label", "marriage_prev",
           "METSCORE_fromPAQ","perE_alco","LTPA_fromPAQ","SNAP_indiv_plus_singleton", 
           "SMK_AVG", "BMI_CLAS", "household_size", "DMDHHSIZ"}
cols_all = [c for c in df.columns if c not in exclude]

# % missing by cycle (split into two lines)
is_na = df[cols_all].isna()
pct_miss = is_na.groupby(df["SDDSRVYR"]).mean().mul(100)

# keep only columns that exceed 80% missing in ANY cycle
pct_miss_gt80 = pct_miss.loc[:, (pct_miss > 80).any(axis=0)].round(1)

print(pct_miss_gt80)


          chol_rx  alcg2
SDDSRVYR                
1.00         0.00  62.90
2.00         0.00  66.50
3.00         0.00  62.90
4.00         0.00  62.70
5.00         0.00  55.40
6.00         0.00  55.40
7.00         0.00  55.10
8.00         0.00  52.20
9.00         0.00  52.80
10.00        0.00  56.00
12.00       29.10  61.40
66.00      100.00  84.30


In [190]:
# df_my_cov_aligned_short[["bmi" , "SEX", "marriage_label"]]

# df_my_cov_aligned_short["chol_rx_structural_missing"]


0         False
1         False
2         False
3         False
4         False
          ...  
128804     True
128805    False
128806    False
128807    False
128808    False
Name: chol_rx_structural_missing, Length: 128809, dtype: bool

In [158]:
# remove duplicate columns

#to_drop = [c for c in ["re2"]
#           if c in df_my_cov_aligned_short.columns]

#df_my_cov_aligned_short = df_my_cov_aligned_short.drop(columns=to_drop)


## save 

In [193]:
from pathlib import Path

OUT = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output")
OUT.mkdir(parents=True, exist_ok=True)

p = OUT / "cov_addv4_99_23.parquet"
df_my_cov_aligned_short.to_parquet(p, index=False)
print("✓ Saved:", p, "shape:", df_my_cov_aligned_short.shape)


✓ Saved: /Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv4_99_23.parquet shape: (128809, 68)
